In [ ]:
import praw
import csv

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id='8YMAaXsl11ShtMeshdQNyg',
    client_secret='wj5FJDoJqIgXZdsfhXXoYLscah3yAw',
    user_agent='Nancy Amandi'
)

# Define the output CSV file
output_file = '/content/reddit_skibidi_toilet_comments.csv'

# Define the header for the CSV file
header = ['post_id', 'subreddit', 'created_utc', 'selftext', 'post_url', 'post_title',
          'link_flair_text', 'score', 'num_comments', 'upvote_ratio',
          'comment_id', 'comment_author', 'comment_body', 'comment_score', 'comment_created_utc']

# Create the CSV file and write the header
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()

    # Search for posts related to "Skibidi Toilet"
    subreddit = reddit.subreddit("all")
    search_results = subreddit.search("Skibidi Toilet", sort='comments', time_filter='all')

    # Extract relevant details from each post and its comments
    for post in search_results:
        print(f"Processing post: {post.id} - {post.title}")  # Debug: Print post info

        try:
            post.comments.replace_more(limit=None)  # To get all comments

            # Iterate over top-level comments
            for comment in post.comments.list():
                post_details = {
                    'post_id': post.id,
                    'subreddit': str(post.subreddit),
                    'created_utc': post.created_utc,
                    'selftext': post.selftext,
                    'post_url': post.url,
                    'post_title': post.title,
                    'link_flair_text': post.link_flair_text,
                    'score': post.score,
                    'num_comments': post.num_comments,
                    'upvote_ratio': post.upvote_ratio,
                    'comment_id': comment.id,
                    'comment_author': str(comment.author),
                    'comment_body': comment.body,
                    'comment_score': comment.score,
                    'comment_created_utc': comment.created_utc
                }

                # Write the post details to the CSV file
                writer.writerow(post_details)

        except Exception as e:
            print(f"An error occurred while processing post: {post.id} - {e}")

print(f"Data extraction complete. Data saved to {output_file}")


# Sentiment Analysis

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

# Load data into a DataFrame
df = pd.read_csv('/content/reddit_skibidi_toilet_comments.csv')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to analyze sentiment
def analyze_sentiment(comment):
    sentiment = analyzer.polarity_scores(comment)
    return sentiment['compound']

# Apply sentiment analysis to the comments column
df['sentiment_score'] = df['comment_body'].apply(analyze_sentiment)

# Select only the necessary columns for the output
sentiment_df = df[['post_id', 'comment_id', 'sentiment_score']]

# Write the sentiment analysis results to a CSV file
sentiment_df.to_csv('/content/sentiment_scores.csv', index=False)
print("Sentiment scores written to 'sentiment_scores.csv'.")

# Now, find the most popular terms in the comments

# Initialize CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the comments to get the term frequencies
term_matrix = vectorizer.fit_transform(df['comment_body'].fillna(''))

# Sum the occurrences of each term
term_frequencies = term_matrix.sum(axis=0)
terms = vectorizer.get_feature_names_out()
term_counts = [(terms[i], term_frequencies[0, i]) for i in range(len(terms))]

# Sort terms by frequency
term_counts_sorted = sorted(term_counts, key=lambda x: x[1], reverse=True)

# Convert the term counts to a DataFrame
terms_df = pd.DataFrame(term_counts_sorted, columns=['term', 'count'])

# Write the term counts to a CSV file
terms_df.to_csv('/content/term_counts.csv', index=False)
print("Term counts written to 'term_counts.csv'.")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Load the data from the CSV file
df = pd.read_csv('/content/reddit_skibidi_toilet_comments.csv')

# Initialize CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the 'selftext' column to get term frequencies
term_matrix = vectorizer.fit_transform(df['selftext'].fillna(''))

# Sum the occurrences of each term
term_frequencies = term_matrix.sum(axis=0)
terms = vectorizer.get_feature_names_out()
term_counts = [(terms[i], term_frequencies[0, i]) for i in range(len(terms))]

# Sort terms by frequency
term_counts_sorted = sorted(term_counts, key=lambda x: x[1], reverse=True)

# Convert the term counts to a DataFrame
terms_df = pd.DataFrame(term_counts_sorted, columns=['term', 'count'])

# Write the term counts to a CSV file
terms_df.to_csv('/content/terms_from_posts.csv', index=False)
print("Term counts from posts written to 'terms_from_posts.csv'.")